## **Подключаем Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import scipy.io
import h5py
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, MaxNLocator
from matplotlib.colors import BoundaryNorm
from PIL import Image
import matplotlib.patches as mpatches

In [ ]:
class ComputingPanel():
  """Класс для рассчета корреляционной панели"""

  def __init__(self, alpha, beta, wave, fs, mu, sigma):
    self.alpha = alpha
    self.beta = beta
    self.wave = wave
    self.fs = fs
    self.mu = mu
    self.sigma = sigma
  
  def gcc_phat(self, sig, refsig, fs=1, max_tau=512, interp=1, freq=0, n=1):
    """Возвращает модуль комплексного числа"""
    if n != None:
      n = sig.shape[0] + refsig.shape[0]
    SIG = np.fft.fft(sig, n=n)
    REFSIG = np.fft.fft(refsig, n=n)
    R = SIG[freq:] * np.conj(REFSIG[freq:])
    cc = np.fft.ifft(R / np.abs(R), interp * n)
    max_shift = int(interp * n / 2)
    if max_tau:
      max_shift = np.minimum(int(interp * fs * max_tau), max_shift)
    cc = np.concatenate((cc[-max_shift:], cc[:max_shift + 1]))
    shift = np.argmax(np.abs(cc)) - max_shift
    tau = shift / float(interp * fs)
    return tau, np.abs(cc)

  def gcc_phat_r(self, sig, refsig, fs=1, max_tau=512, interp=1, freq=50, n=1):
    """Возвращает реальную часть комплексного числа"""
    if n != None:
        n = sig.shape[0] + refsig.shape[0]
    SIG = np.fft.rfft(sig, n=n)
    REFSIG = np.fft.rfft(refsig, n=n)
    R = SIG[:freq] * np.conj(REFSIG[:freq])
    cc = np.fft.irfft(R / np.abs(R), interp * n)
    max_shift = int(interp * n / 2)
    if max_tau:
        max_shift = np.minimum(int(interp * fs * max_tau), max_shift)
    cc = np.concatenate((cc[-max_shift:], cc[:max_shift + 1]))
    shift = np.argmax(np.abs(cc)) - max_shift
    tau = shift / float(interp * fs)
    return tau, cc

  def delay_compute(self, r, sens_pos, sour_pos, delta_sens, c = 1450):
    """Считает задержку(геометрически)"""
    s_1 = np.sqrt(r**2 + (sens_pos - sour_pos)**2)
    s_2 = np.sqrt(r**2 + (sens_pos - sour_pos + delta_sens)**2)
    tau = (s_1 - s_2) / c
    return tau
  
  def cross_count(self, k1, k2, k3, tau, complexs = True):
    """Считает модельную задержку"""
    attenuation = np.exp(-self.alpha * np.sqrt((k1 - k2) ** 2 + k3 ** 2))
    sig = (self.wave*attenuation + self.beta * np.random.normal(self.mu, self.sigma, len(self.wave)))
    refsig = (np.roll(self.wave, int(tau * self.fs)) * attenuation + 
              self.beta * np.random.normal(self.mu, self.sigma, len(self.wave)))
    if complexs:
      tau, cc = self.gcc_phat(sig = sig, refsig = refsig)
    else:
      tau, cc = self.gcc_phat_r(sig = sig, refsig = refsig)
    return tau, cc
  
  def on_each_point(self, deep_range, pair_of_sensor, sourse_form, flow = False):
    kesy = sorted(pair_of_sensor.keys())
    cross_corr_for_each_pair_of_sensor=[]
    for key in kesy:
      values = pair_of_sensor[key]
      cross_corr = []
      for sours_pos in sourse_form:
        cross_form = []
        for sens_pos in deep_range:
          tau = self.delay_compute(r = sours_pos[1],
                                 sens_pos = sens_pos,
                                 sour_pos = sours_pos[0],
                                 delta_sens = values)
          delay, cc = self.cross_count(k1 = sens_pos, 
                                       k2 = sours_pos[0],
                                       k3 = sours_pos[1],
                                       tau = tau)
          if flow == True:
            cc_new = []
            ma = max(cc)
            st = np.std(cc)
            ind = cc.tolist().index(max(cc))
            shift = int((ind-len(cc)/2)/1.5)
            print(ind, shift)
            cc = np.roll(cc, -shift)
            for k in cc:
              if k > ma/2-ma/7:
                cc_new.append(st)
              else:
                cc_new.append(k)
            cc = np.array(cc_new)
          else:
            cc = np.array(cc)
          cross_form.append(cc)
        cross_form = np.array(cross_form)
        cross_corr.append(cross_form)
      cross_corr = np.array(cross_corr)
      cross_corr_for_each_pair_of_sensor.append(np.mean(np.array(cross_corr), axis=0))
    return np.array(cross_corr_for_each_pair_of_sensor)

In [ ]:
class VisualizePanel():
  """Класс отвечающий за визуализацию панелей"""
  def func_picture(self, arrays_list, Fs = 1, x_tick=None, y_tick=None,
                     x_size=15, y_size=10, tick_size=10,
                     name_size=20, name_oy_size=20, name_ox_size=20, file=None,
                     name_main=None, name_oy=None, name_ox=None, look_picture = False):
        """
        @param arrays_list: список временных рядов
        @param x_tick: шаг по оси Ох 
        @param y_tick: шаг по оси Oy
        @param x_size: размер графика по Ох
        @param y_size: размер графика по Оу
        @param tick_size: размер подписей отсчетов по осям
        @param name_size: размер подписи названия графика
        @param name_oy_size: размер подписи названия оси Оу
        @param name_ox_size: размер подписи названия оси Ох
        @param file: путь (D://названию.png)
        @param name_main: название графика
        @param name_oy: название оси Оу
        @param name_ox: назание оси Ох
        """
        fig, ax = plt.subplots(figsize=(x_size, y_size))
        for k_y in arrays_list:
            k_x = np.arange(0, len(k_y)/Fs,1/Fs)
            ax.plot(k_x, k_y)
            xax = ax.xaxis
            yax = ax.yaxis
            xlab = xax.get_ticklabels()
            ylab = yax.get_ticklabels()
            if ((x_tick != None) or (y_tick != None)):
                xmajorFormatter = FormatStrFormatter('%d')
                ymajorFormatter = FormatStrFormatter('%0.3f')
                xmajorLocator = MultipleLocator(x_tick)
                ymajorLocator = MultipleLocator(y_tick)
                xax.set_major_locator(xmajorLocator)
                xax.set_major_formatter(xmajorFormatter)
                yax.set_major_locator(ymajorLocator)
                yax.set_major_formatter(ymajorFormatter)
            yax.grid(True, which='major', color='grey', alpha=0.5)

            for label in xlab:
                label.set_fontsize(tick_size)
                #label.set_color('blue')
                label.set_rotation(tick_size)
            for label in ylab:
                label.set_fontsize(tick_size)
            ax.set_title(label=name_main, fontsize=name_size)
            ax.set_ylabel(ylabel=name_oy, fontsize=name_oy_size)
            ax.set_xlabel(xlabel=name_ox, fontsize=name_ox_size)
            ax.grid(True, which='major', color='grey', linestyle='dashed', alpha=0.5)
        if file is not None:
            fig.savefig(file)
        if look_picture is True:
            plt.show()
            plt.close()

  def crop(self, image_path, coords, saved_location):
        """
        @param image_path: The path to the image to edit
        @param coords: A tuple of x/y coordinates (x1, y1, x2, y2)
        @param saved_location: Path to save the cropped image
        """
        image_obj = Image.open(image_path)
        cropped_image = image_obj.crop(coords)
        cropped_image.save(saved_location)

  def plot_panel(self, corr_list, deep_range, fname, max_v, info = True, ticks = False,
                   title = '', xlabel = 'задержка, с', ylabel = 'глубина, м',
                   crop = False, coords = (61, 35, 450, 267), figsize_x = 10, figsize_y = 5, legend = None, my_dpi = None):
      """
      Визуализация корреляционной панели:

      @param info: включить названия по осям
      @param ticks: отключить разметку по осям
      @param crop: обрезать изображение
      """
      deep_list = deep_range.tolist()
      Fs = 117187.5
      z = np.array(corr_list)
      y = (np.array([deep_list] * len(corr_list[0]))).transpose()
      x = np.array([[k / Fs for k in range(-int(len(y[0]) / 2), int(len(y[0]) / 2) + 1)]] * len(y))
      print(len(x), len(x[0]), len(y), len(y[0]))
      z = z[:-1, :-1]
      #levels = MaxNLocator(nbins=500).tick_values(z.min(), z.max())
      levels = MaxNLocator(nbins=500).tick_values(0, max_v)
      cmap = plt.get_cmap('Spectral_r', 1000)
      norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
      fig, ax0 = plt.subplots(figsize=(figsize_x, figsize_y))
      im = ax0.pcolormesh(x, y, z, cmap=cmap, norm=norm)
      if info:
        fig.colorbar(im, ax=ax0)
        ax0.set_title(title)
        ax0.set_xlabel(xlabel, fontsize = 10)
        ax0.set_ylabel(ylabel, fontsize = 10)
        #fig = plt.gca()
        #fig1 = plt.gcf()
      if ticks:
        plt.xticks([])
        plt.yticks([])
      if legend != None:
        line,  = ax0.plot([], label = legend, linestyle = 'None')
        ax0.legend(fontsize = 14, facecolor = 'white', edgecolor = 'None',
                   markerfirst=False, loc = 'upper left' )
      if my_dpi != None:
        plt.savefig(fname=fname, dpi = my_dpi)
      else:
        plt.savefig(fname=fname)
      #plt.show()
      if crop:
        self.crop(image_path=fname, coords=coords, saved_location=fname)
      plt.close()

  def spectrogramm_real(self, cross_corr,deep_range,fname,max_v=None, min_v=None, chast = None):
      """Построить спектрограмму"""
      fft_list = {}
      i = 0
      for k in cross_corr:
          fft_list.update({deep_range[i]: np.fft.rfft(k * np.kaiser(len(k), 8))})
          i += 1
      print('Строиться спектр')
      z = np.array([abs(k) for k in fft_list.values()])
      m = 0
      n = 0
      l = 0
      print(np.array(z).shape)
      for k in z:
        if k.max() > m:
          m = k.max()
          l = n
        n += 1
      print(m, l)
      if max_v == None:
        max_v = 1
        min_v = 0
        z = z/m
      y = (np.array([sorted([float(k) for k in list(fft_list.keys())], reverse=True)] * 
                    len(fft_list[deep_range[0]]))).transpose()
      x = np.array([[k for k in range(0, int(len(y[0])))]] * len(y))

      if chast != None:
        x = np.array([k[:chast] for k in x])
        y = np.array([k[:chast] for k in y])
        z = np.array([k[:chast] for k in z])
      print(len(x), len(x[0]), len(y), len(y[0]), len(z), len(z[0]))

      z = z[:-1, :-1]
      levels = MaxNLocator(nbins=100).tick_values(min_v, max_v)
      cmap = plt.get_cmap('Spectral_r', 1000)
      norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

      fig, ax0 = plt.subplots(figsize=(7, 5))
      im = ax0.pcolormesh(x, y, z, cmap=cmap, norm=norm)
      fig.colorbar(im, ax=ax0)
      """
      xax = ax0.xaxis
      yax = ax0.yaxis
      xlab = xax.get_ticklabels()
      ylab = yax.get_ticklabels()
      tick_size = 3
      for label in xlab:
        label.set_fontsize(tick_size)
      for label in ylab:
        label.set_fontsize(tick_size)
      """
      #ax0.tick_params(direction='out', labelsize = 5)
      ax0.set_title(f' ')
      ax0.set_xlabel('частота, кГц', fontsize=12)
      ax0.set_ylabel('глубина, м', fontsize=12)
      
      plt.savefig(fname)
      plt.close()
      return l
    
  def spectrogramm_max_value(self, cross_corr,deep_range):
    """Посчитать максимальное значение мощности"""
    fft_list = {}
    i = 0
    for k in cross_corr:
        fft_list.update({deep_range[i]: np.fft.rfft(k * np.kaiser(len(k), 8))})
        i += 1
    print('Строиться спектр')
    z = np.array([abs(k) for k in fft_list.values()])
    
    m = 0
    n = 0
    l = 0
    print(np.array(z).shape)
    for k in z:
      if k.max() > m:
        m = k.max()
        l = n
      n += 1
    print(m, l)
    return l

# **Задаем параметры источника**

**Задаем форму источника**


In [ ]:
"""Создаем словарь источников с разлиными формами"""
#первое значение это от ствола скважины, второе по глубине
form_point_array = np.array([[0, 0.0]])
forms_list = {
    'point2': [[np.arange(0.0, 0.03, 0.01), np.arange(0.00, 0.03, 0.01)]],                  #2см*2cm
    'point5': [[np.arange(0.0, 0.06, 0.01), np.arange(-0.02, 0.04, 0.01)]],                 #5cm*5cm
    'point10': [[np.arange(0.0, 0.11, 0.01), np.arange(-0.05, 0.06, 0.01)]],                #10cm*10cm
    'plast1': [[np.arange(0.0, 0.5, 0.1), np.arange(-0.5, 0.6, 0.1)]],                      #1 вид: 1м * 0.5м
    'plast2': [[np.arange(0.0, 1.0, 0.1), np.arange(0, 0.1, 0.01)]],                        #2 вид: 0.1м * 1м
    'plast3': [[np.arange(0.0, 1.0, 0.1), np.arange(-0.5, 0.6, 0.1) ]],                     #3 вид: 1м * 1м
    'plast4': [[np.arange(0.0, 1.0, 0.1), np.arange(-0.25, 0.25, 0.1)]],                    #4 вид: 0.5 * 1м
    'line1': [[np.arange(0.0, 0.3, 0.1), np.arange(-0.5, 0.6, 0.1)]],                       #1 вид: 1м * 0.2м
    'line2': [[np.arange(0.0, 1.0, 0.1), np.arange(-0.25, 0.25, 0.1)]],                     #2 вид: 0.5м * 0.2м
    'line3': [[np.arange(0.0, 0.2, 0.1), np.arange(-0.5, 0.6, 0.1)]],                       #3 вид: 1м * 0.1м
    'line4': [[np.arange(0.0, 0.2, 0.1), np.arange(-0.25, 0.25, 0.1)]],                     #4 вид: 0.5м * 0.1м
}
form_names = list(forms_list.keys())

In [ ]:
form_list = []
index_name =0 #здесь выбор источника
for array in forms_list[form_names[index_name]]:
  for dist in array[0]:
    form_list = form_list + [[dist, deep] for deep in array[1]]
form_array = np.array(form_list)

**Пропишем общий путь к каталогу**

In [ ]:
path = '/content/drive/My Drive/MagistracyProject'

**Задаем остальные параметры источника**

In [ ]:
a = -10.0
b =  10.1
d  = 0.2
deep_range = np.arange(-10, 10.1, 0.1)            # шаг датч.
radian_range = np.arange(0.2, 0.4, 0.05)          # позиц. источн. от ствола
sourse_range = np.array([0.0])                    # позиц. источн. по глубине
alpha = 0.0                                       # коэф. ослабл сигнала
beta = 50                                         # коэф. усил шума
mu = 0                                            # мат. ожид 
sigma = 0.5                                       # дисперс. 
fs = 117187.5                                     # частота дискр.
N = 10                                            # во сколько раз расстояние между источниками увеличить, т.е размер 
ADC = 2**15                                       # разрядн ацп
pair_of_sensor = {'12' : 0.3}

**Выбор аудиофрейма**

In [ ]:
df = pd.read_json(os.path.join(path, 'frames.json'))
df_flow = df[df.type == 'flow']
df_point = df[df.type == 'point']

# **Для разовых тестов**

In [ ]:
deep_range = np.arange(-10.0, 10.1, 0.5)            # шаг датч.
radian_range = np.array([0.2])          # позиц. источн. от ствола
sourse_range = np.array([0.0])                    # позиц. источн. по глубине
alpha = 0.2                                     # коэф. ослабл сигнала
beta = 70                                    # коэф. усил шума
mu = 0                                            # мат. ожид 
sigma = 1                                       # дисперс. 
fs = 117187.5                                     # частота дискр.
N = 10                                            # во сколько раз расстояние между источниками увеличить, т.е размер 
ADC = 2**15                                       # разрядн ацп
pair_of_sensor = {'12' : 0.3}

In [ ]:
wave = np.array(df_point.iloc[0]['frame'])

In [ ]:
path_model = os.path.join(path, 'pres')
if not os.path.exists(path_model):
    os.makedirs(path_model)
print(path_model)
x = 454 
y = 571
cp = ComputingPanel(alpha, beta, wave, fs, mu, sigma)
for rad in radian_range:
  cross_corr_list = []
  path_rad = os.path.join(path_model, 'rad {}'.format(round(rad,3)))
  print(path_rad)
  if not os.path.exists(path_rad):
    os.makedirs(path_rad)
  for sour in sourse_range:
    form = np.array([[d[0] + sour, d[1] + rad] for d in form_array])
    cross_corr = cp.on_each_point(deep_range, pair_of_sensor, form, flow = False)
    cross_corr_list.append(cross_corr)

  i = 0
  for k in cross_corr_list:
    ds = np.array(k[0])
    d = [k[x:y] for k in ds]
    
    vp = VisualizePanel()
    fname = os.path.join(path_rad, str(i) + '.png')
    fname_s = os.path.join(path_rad, str(i) + 's.png')
    fname_csv = os.path.join(path_rad, str(i) + '.csv')
    print(fname)
    #l = vp.spectrogramm_real(d, deep_range,fname_s, max_v = None, min_v = 0, chast = 30)
    l = vp.spectrogramm_max_value(d, deep_range)
    
    #обрезка по спектру
    obrez = False
    if obrez:
      sl = slice(l-25, l+25)
      d = d[sl]
      deep_range_new = deep_range[sl]

    d_arr = np.array(d, dtype=np.float16)
    print(d_arr.shape)
    pd.DataFrame(d_arr).to_csv(fname_csv)

    if not obrez:
      deep_range_new = deep_range 

    vp.plot_panel(d_arr, deep_range_new, fname, max_v = 0.1, figsize_x = 10, figsize_y = 7,
                  ticks = False, info = True, crop = False, coords = (28,18,161,126), legend = None, my_dpi = None)
  
    del vp
    i += 1
del cp

/content/drive/My Drive/MagistracyProject/pres
/content/drive/My Drive/MagistracyProject/pres/rad 0.2
/content/drive/My Drive/MagistracyProject/pres/rad 0.2/0.png
Строиться спектр
(41, 59)
1.8427267137687244 20
(41, 117)
41 117 41 117


# **Генератор распределенных источников**

In [ ]:
import shutil
shutil.rmtree('/content/drive/My Drive/MagistracyProject/modeling/wave0')

In [ ]:
deep_range = np.arange(-10.0, 10.1, 0.2)            # шаг датч.
radian_range = np.arange(0.2, 1.0, 0.1)          # позиц. источн. от ствола
sourse_range = np.array([0.0])                    # позиц. источн. по глубине
alpha_arr = np.arange(0.2, 0.4, 0.1)                                     # коэф. ослабл сигнала
beta_arr = np.arange(50, 125, 25)                                    # коэф. усил шума
mu = 0                                            # мат. ожид 
sigma = 1                                         # дисперс. 
fs = 117187.5                                     # частота дискр.
N = 10                                            # во сколько раз расстояние между источниками увеличить, т.е размер 
ADC = 2**15                                       # разрядн ацп
pair_of_sensor = {'12' : 0.3}

In [ ]:
2**15

32768

In [ ]:
print(alpha_arr.shape, beta_arr.shape, radian_range.shape, df_flow.shape)
print(alpha_arr.shape[0]*beta_arr.shape[0]*radian_range.shape[0]*63*7)

(7,) (10,) (8,) (5, 2)
8400


In [ ]:
np.arange(3,11)

array([ 3,  4,  5,  6,  7,  8,  9, 10])

In [ ]:
%%time
path_model = os.path.join(path, 'modeling')

x = 454 
y = 571
for num_wave in np.arange(0, 60):
  wave = np.array(df_flow.iloc[num_wave]['frame'])
  for form_index in range(3,11):
    form_list = []
    index_name = form_index #здесь выбор источника c 3 распределенные
    for array in forms_list[form_names[index_name]]:
      for dist in array[0]:
        form_list = form_list + [[dist, deep] for deep in array[1]]
    form_array = np.array(form_list)

    for alpha in alpha_arr:
      for beta in beta_arr:
        cp = ComputingPanel(alpha, beta, wave, fs, mu, sigma)
        for rad in radian_range:
          try:
            deep_range = np.arange(-10.0, 10.1, 0.2)
            cross_corr_list = []
            path_rad = os.path.join(path_model, 'wave{}/form{}/rad {}/a {}/b {}'.format(num_wave, form_index, round(rad,3), round(alpha, 3), beta))
            print(path_rad)
            if not os.path.exists(path_rad):
              os.makedirs(path_rad)
            for sour in sourse_range:
              form = np.array([[d[0] + sour, d[1] + rad] for d in form_array])
              cross_corr = cp.on_each_point(deep_range, pair_of_sensor, form, flow = False)
              cross_corr_list.append(cross_corr)

            i = 0
            for k in cross_corr_list:
              ds = np.array(k[0])
              d = [k[x:y] for k in ds]
              
              vp = VisualizePanel()
              fname = os.path.join(path_rad, str(i) + '.png')
              fname_s = os.path.join(path_rad, str(i) + 's.png')
              fname_csv = os.path.join(path_rad, str(i) + '.csv')
              print(fname)
              #l = vp.spectrogramm_real(d, deep_range,fname_s, max_v = None, min_v = 0, chast = 30)
              l = vp.spectrogramm_max_value(d, deep_range)
              
              #обрезка по спектру
              obrez = True
              if obrez:
                sl = slice(l-25, l+25)
                d = d[sl]
                deep_range_new = deep_range[sl]

              d_arr = np.array(d, dtype=np.float16)
              print(d_arr.shape)
              pd.DataFrame(d_arr).to_csv(fname_csv)

              if not obrez:
                deep_range_new = deep_range 

              vp.plot_panel(d_arr, deep_range_new, fname, max_v = 0.1, figsize_x = 3, figsize_y = 2,
                            ticks = False, info = True, crop = True, coords = (28,18,161,126), legend = None, my_dpi = None)
            
              del vp
              i += 1
          except: pass
        del cp

# **Генератор точечных источников**

In [ ]:
deep_range = np.arange(-10.0, 10.1, 0.2)            # шаг датч.
radian_range = np.arange(0.2, 1.0, 0.1)          # позиц. источн. от ствола
sourse_range = np.array([0.0])                    # позиц. источн. по глубине
alpha_arr = np.arange(0.1, 0.4, 0.05)                                     # коэф. ослабл сигнала
beta_arr = np.arange(50, 150, 10)                                    # коэф. усил шума
mu = 0                                            # мат. ожид 
sigma = 1                                         # дисперс. 
fs = 117187.5                                     # частота дискр.
N = 10                                            # во сколько раз расстояние между источниками увеличить, т.е размер 
ADC = 2**15                                       # разрядн ацп
pair_of_sensor = {'12' : 0.3}

In [ ]:
print(alpha_arr.shape, beta_arr.shape, radian_range.shape, df_point.shape)
print(alpha_arr.shape[0]*beta_arr.shape[0]*radian_range.shape[0]*5*3)

(7,) (10,) (8,) (5, 2)
8400


In [ ]:
%%time
path_model = os.path.join(path, 'modeling/point')

x = 454 
y = 571
for num_wave in np.arange(0, 6):
  wave = np.array(df_point.iloc[num_wave]['frame'])
  for form_index in range(0, 3):
    form_list = []
    index_name = form_index #здесь выбор источника c 3 распределенные
    for array in forms_list[form_names[index_name]]:
      for dist in array[0]:
        form_list = form_list + [[dist, deep] for deep in array[1]]
    form_array = np.array(form_list)

    for alpha in alpha_arr:
      for beta in beta_arr:
        cp = ComputingPanel(alpha, beta, wave, fs, mu, sigma)
        for rad in radian_range:
          try:
            deep_range = np.arange(-10.0, 10.1, 0.2)
            cross_corr_list = []
            path_rad = os.path.join(path_model, 'wave{}/form{}/rad {}/a {}/b {}'.format(num_wave, form_index, round(rad,3), round(alpha, 3), beta))
            print(path_rad)
            if not os.path.exists(path_rad):
              os.makedirs(path_rad)
            for sour in sourse_range:
              form = np.array([[d[0] + sour, d[1] + rad] for d in form_array])
              cross_corr = cp.on_each_point(deep_range, pair_of_sensor, form, flow = False)
              cross_corr_list.append(cross_corr)

            i = 0
            for k in cross_corr_list:
              ds = np.array(k[0])
              d = [k[x:y] for k in ds]
              
              vp = VisualizePanel()
              fname = os.path.join(path_rad, str(i) + '.png')
              fname_s = os.path.join(path_rad, str(i) + 's.png')
              fname_csv = os.path.join(path_rad, str(i) + '.csv')
              print(fname)
              #l = vp.spectrogramm_real(d, deep_range,fname_s, max_v = None, min_v = 0, chast = 30)
              l = vp.spectrogramm_max_value(d, deep_range)
              
              #обрезка по спектру
              obrez = True
              if obrez:
                sl = slice(l-25, l+25)
                d = d[sl]
                deep_range_new = deep_range[sl]

              d_arr = np.array(d, dtype=np.float16)
              print(d_arr.shape)
              pd.DataFrame(d_arr).to_csv(fname_csv)

              if not obrez:
                deep_range_new = deep_range 

              vp.plot_panel(d_arr, deep_range_new, fname, max_v = 0.1, figsize_x = 3, figsize_y = 2,
                            ticks = False, info = True, crop = True, coords = (28,18,161,126), legend = None, my_dpi = None)
            
              del vp
              i += 1
          except: pass
        del cp

# **Генератор идеальных точечных данных!**

In [ ]:
deep_range = np.arange(-10.0, 10.1, 0.2)            # шаг датч.
radian_range = np.arange(0.2, 1.0, 0.1)          # позиц. источн. от ствола
sourse_range = np.array([0.0])                    # позиц. источн. по глубине
alpha_arr = np.arange(0.0, 0.4, 0.05)                                     # коэф. ослабл сигнала
beta_arr = np.arange(0, 150, 10)                                    # коэф. усил шума
mu = 0                                            # мат. ожид 
sigma = 1                                         # дисперс. 
fs = 117187.5                                     # частота дискр.
N = 10                                            # во сколько раз расстояние между источниками увеличить, т.е размер 
ADC = 2**15                                       # разрядн ацп
pair_of_sensor = {'12' : 0.3}

In [ ]:
%%time
path_model = os.path.join(path, 'modeling/point')

x = 454 
y = 571
for num_wave in np.arange(0, 6):
  wave = np.array(df_point.iloc[num_wave]['frame'])
  for form_index in range(0, 3):
    form_list = []
    index_name = form_index #здесь выбор источника c 3 распределенные
    for array in forms_list[form_names[index_name]]:
      for dist in array[0]:
        form_list = form_list + [[dist, deep] for deep in array[1]]
    form_array = np.array(form_list)

    for alpha in alpha_arr[:1]:
      for beta in beta_arr[:1]:
        cp = ComputingPanel(alpha, beta, wave, fs, mu, sigma)
        for rad in radian_range:
          try:
            deep_range = np.arange(-10.0, 10.1, 0.2)
            cross_corr_list = []
            path_rad = os.path.join(path_model, 'wave{}/form{}/rad {}/a {}/b {}'.format(num_wave, form_index, round(rad,3), round(alpha, 3), beta))
            print(path_rad)
            if not os.path.exists(path_rad):
              os.makedirs(path_rad)
            for sour in sourse_range:
              form = np.array([[d[0] + sour, d[1] + rad] for d in form_array])
              cross_corr = cp.on_each_point(deep_range, pair_of_sensor, form, flow = False)
              cross_corr_list.append(cross_corr)

            i = 0
            for k in cross_corr_list:
              ds = np.array(k[0])
              d = [k[x:y] for k in ds]
              
              vp = VisualizePanel()
              fname = os.path.join(path_rad, str(i) + '.png')
              fname_s = os.path.join(path_rad, str(i) + 's.png')
              fname_csv = os.path.join(path_rad, str(i) + '.csv')
              print(fname)
              #l = vp.spectrogramm_real(d, deep_range,fname_s, max_v = None, min_v = 0, chast = 30)
              l = vp.spectrogramm_max_value(d, deep_range)
              
              #обрезка по спектру
              obrez = True
              if obrez:
                sl = slice(l-25, l+25)
                d = d[sl]
                deep_range_new = deep_range[sl]

              d_arr = np.array(d, dtype=np.float16)
              print(d_arr.shape)
              pd.DataFrame(d_arr).to_csv(fname_csv)

              if not obrez:
                deep_range_new = deep_range 

              vp.plot_panel(d_arr, deep_range_new, fname, max_v = 0.1, figsize_x = 3, figsize_y = 2,
                            ticks = False, info = True, crop = True, coords = (28,18,161,126), legend = None, my_dpi = None)
            
              del vp
              i += 1
          except: pass
        del cp